Задание 1. Базовое изучение
Изучить представленный набор данных на основе описания его столбцов и выбрать 8 столбцов для дальнейшего изучения (среди них должны быть как числовые, так и категориальные). Провести расчет базовых метрик для них, кратко описать результаты.


In [1]:
import numpy as np
import pandas as pd

In [19]:
data = pd.read_csv('datahorses.csv', header=None, na_values='?')
df = data[[(1),(2),(4),(7),(9),(11),(12),(14),(3)]]
df.columns = ['Age', 'Hospital Number', 'pulse', 'capillary refill time','temperature of extremities', 'peristalsis', 'abdominal distension',
           'nasogastric reflux ','rectal temperature']
df[:5]


,Age,Hospital Number,pulse,capillary refill time,temperature of extremities,peristalsis,abdominal distension,nasogastric reflux,rectal temperature
0,1,530101,66.0,3.0,2.0,4.0,4.0,NaN,38.5
1,1,534817,88.0,NaN,1.0,4.0,2.0,NaN,39.2
2,1,530334,40.0,1.0,1.0,3.0,1.0,NaN,38.3
3,9,5290409,164.0,1.0,2.0,4.0,4.0,2.0,39.1
4,1,530255,104.0,NaN,2.0,NaN,NaN,NaN,37.3


In [4]:
df.describe()

,Age,Hospital Number,pulse,capillary refill time,temperature of extremities,peristalsis,abdominal distension,nasogastric reflux,rectal temperature
count,300.000000,3.000000e+02,276.000000,231.000000,268.000000,256.000000,244.000000,194.000000,240.000000
mean,1.640000,1.085889e+06,71.913043,2.017316,1.305970,2.917969,2.266393,1.582474,38.167917
std,2.173972,1.529801e+06,28.630557,1.042428,0.477629,0.976744,1.065131,0.805359,0.732289
min,1.000000,5.184760e+05,30.000000,1.000000,1.000000,1.000000,1.000000,1.000000,35.400000
25%,1.000000,5.289040e+05,48.000000,1.000000,1.000000,3.000000,1.000000,1.000000,37.800000
50%,1.000000,5.303055e+05,64.000000,2.000000,1.000000,3.000000,2.000000,1.000000,38.200000
75%,1.000000,5.347275e+05,88.000000,3.000000,2.000000,4.000000,3.000000,2.000000,38.500000
max,9.000000,5.305629e+06,184.000000,4.000000,3.000000,4.000000,4.000000,3.000000,40.800000


Задание 2. Работа с выбросами
В выбранных числовых столбцах найти выбросы, выдвинуть гипотезы об их причинах и проинтерпретировать результаты. Принять и обосновать решение о дальнейшей работе с ними.

In [134]:
# найдем межквартильный размах
q1_p = df['pulse'].quantile(0.25)
q3_p = df['pulse'].quantile(0.75)
IQR_p = q3_p - q1_p
lower_bound_p = q1_p - (IQR_p*1.5)
upper_bound_p = q3_p + (IQR_p*1.5)
remove_outliers_p = df[df['pulse'].between(lower_bound_p,upper_bound_p, inclusive = True )]

# выбросы (объединяем исходный датафрейм с датафреймом межквартильного размаха и удаляем дубликаты )
remove_outliers_p = pd.concat([df, remove_outliers_p]).drop_duplicates(keep=False)
# для наглядности уберем строки с NaN
remove_outliers_p = remove_outliers_p.dropna(subset = ['pulse'])
remove_outliers_p['pulse']

3      164.0
41     150.0
55     160.0
255    184.0
275    150.0
Name: pulse, dtype: float64

In [135]:
# найдем межквартильный размах
q1_t = df['rectal temperature'].quantile(0.25)
q3_t = df['rectal temperature'].quantile(0.75)
IQR_t = q3_t - q1_t
lower_bound_t = q1_t - (IQR_t*1.5)
upper_bound_t = q3_t + (IQR_t*1.5)
remove_outliers_t = df[df['rectal temperature'].between(lower_bound_t,upper_bound_t, inclusive = True )]

# выбросы (объединяем исходный датафрейм с датафреймом межквартильного размаха и удаляем дубликаты и значения Nan )
remove_outliers_t = pd.concat([df, remove_outliers_t]).drop_duplicates(keep=False).dropna(subset = ['rectal temperature'])
remove_outliers_t['rectal temperature']

20     39.9
44     35.4
75     39.7
80     36.4
99     39.6
118    36.5
141    36.0
238    36.1
251    36.6
259    40.8
281    40.0
298    36.5
Name: rectal temperature, dtype: float64

Выбросы по температуре и пульсу скорее всего не ошибочны, пульс и температура тела индивидуальны в каждом отдельном случае. Данные оставляем.

Задание 3. Работа с пропусками
Рассчитать количество пропусков для всех выбранных столбцов. Принять и обосновать решение о методе работы с пропусками по каждому столбцу, сформировать датафрейм, в котором пропуски будут отсутствовать.

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Age                         300 non-null    int64  
 1   Hospital Number             300 non-null    int64  
 2   pulse                       276 non-null    float64
 3   capillary refill time       231 non-null    float64
 4   temperature of extremities  268 non-null    float64
 5   peristalsis                 256 non-null    float64
 6   abdominal distension        244 non-null    float64
 7   nasogastric reflux          194 non-null    float64
 8   rectal temperature          240 non-null    float64
dtypes: float64(7), int64(2)
memory usage: 21.2 KB


In [10]:
# посчитаем пропуски в %
(df.isna().mean()*100).round(2)

Age                            0.00
Hospital Number                0.00
pulse                          8.00
capillary refill time         23.00
temperature of extremities    10.67
peristalsis                   14.67
abdominal distension          18.67
nasogastric reflux            35.33
rectal temperature            20.00
dtype: float64

очень много пропусков у nasogastric reflux, удалять их или нет решение зависит от того на сколько этот симптом является важным для тех или иных выводов..я не знаю))) Треть пропущена, это много, удалим. 

In [48]:
df.drop(['nasogastric reflux '], axis=1).head()

,Age,Hospital Number,pulse,capillary refill time,temperature of extremities,peristalsis,abdominal distension,rectal temperature
0,1,530101,66.0,3.0,2.0,4.0,4.0,38.5
1,1,534817,88.0,NaN,1.0,4.0,2.0,39.2
2,1,530334,40.0,1.0,1.0,3.0,1.0,38.3
3,9,5290409,164.0,1.0,2.0,4.0,4.0,39.1
4,1,530255,104.0,NaN,2.0,NaN,NaN,37.3


In [32]:
print('pulse:')
print('среднее:', df['pulse'].mean())
print('стандартное отклонение:', df['pulse'].std())


pulse:
среднее: 71.91304347826087
стандартное отклонение: 28.630556660735003


In [9]:
print(df['pulse'].describe())
#отклонение от среднего приличное плюс оставляем выбросы, поэтому заменим медианой
df_fillna_p = df.copy()
df_fillna_p['pulse'].fillna(df['pulse'].median(), inplace=True)
print('------------------------------')
print(df_fillna_p['pulse'].describe())
print('среднее:', df_fillna_p['pulse'].mean())

count    276.000000
mean      71.913043
std       28.630557
min       30.000000
25%       48.000000
50%       64.000000
75%       88.000000
max      184.000000
Name: pulse, dtype: float64
------------------------------
count    300.000000
mean      71.280000
std       27.541545
min       30.000000
25%       48.000000
50%       64.000000
75%       88.000000
max      184.000000
Name: pulse, dtype: float64
среднее: 71.28


In [143]:
print(df.groupby('Age')['pulse'].median()) # можно предположить, что чем старше лошадь тем пульс выше
age_pulse = df.copy()
age_pulse['pulse'].fillna(df.groupby('Age')['pulse'].transform('median'), inplace=True)
age_pulse['pulse'].describe()
# данные полученные в разрезе возраста немного точнее чем без его учета.

Age
1     60.0
9    124.0
Name: pulse, dtype: float64


count    300.000000
mean      71.386667
std       27.968278
min       30.000000
25%       48.000000
50%       60.000000
75%       88.000000
max      184.000000
Name: pulse, dtype: float64

In [14]:
print('rectal temperature:')
print('среднее:', df['rectal temperature'].mean())
print('стандартное отклонение:', df['rectal temperature'].std())

rectal temperature:
среднее: 38.16791666666669
стандартное отклонение: 0.7322886641121578


In [144]:
print(df['rectal temperature'].describe())

df_fillna_t = age_pulse
df_fillna_t['rectal temperature'].fillna(age_pulse['rectal temperature'].median(), inplace=True)
print('------------------------------')
print(df_fillna_t['rectal temperature'].describe())
print('среднее:', df_fillna_t['rectal temperature'].mean())


count    240.000000
mean      38.167917
std        0.732289
min       35.400000
25%       37.800000
50%       38.200000
75%       38.500000
max       40.800000
Name: rectal temperature, dtype: float64
------------------------------
count    300.000000
mean      38.174333
std        0.654831
min       35.400000
25%       37.900000
50%       38.200000
75%       38.500000
max       40.800000
Name: rectal temperature, dtype: float64
среднее: 38.17433333333336


In [119]:
df_fillna_cap = df_fillna_t
df_fillna_cap['capillary refill time'].fillna(df_fillna_t['capillary refill time'].mode()[0], inplace = True)

In [120]:
df_fillna_te = df_fillna_cap
df_fillna_te['temperature of extremities'].fillna(df_fillna_cap['temperature of extremities'].mode()[0], inplace = True)

In [121]:
df_fillna_per = df_fillna_te
df_fillna_per['peristalsis'].fillna(df_fillna_te['peristalsis'].mode()[0], inplace = True)

In [145]:
df_fillna_abd = df_fillna_per
df_fillna_abd['abdominal distension'].fillna(df_fillna_per['abdominal distension'].mode()[0], inplace = True)
df_fillna_abd.drop(['nasogastric reflux '], axis=1).head(10)

,Age,Hospital Number,pulse,capillary refill time,temperature of extremities,peristalsis,abdominal distension,rectal temperature
0,1,530101,66.0,3.0,2.0,4.0,4.0,38.5
1,1,534817,88.0,1.0,1.0,4.0,2.0,39.2
2,1,530334,40.0,1.0,1.0,3.0,1.0,38.3
3,9,5290409,164.0,1.0,2.0,4.0,4.0,39.1
4,1,530255,104.0,1.0,2.0,3.0,1.0,37.3
5,1,528355,64.0,1.0,1.0,3.0,2.0,38.2
6,1,526802,48.0,1.0,1.0,3.0,3.0,37.9
7,1,529607,60.0,1.0,1.0,4.0,2.0,38.2
8,1,530051,80.0,4.0,1.0,4.0,4.0,38.2
9,9,5299629,90.0,1.0,1.0,3.0,1.0,38.3


In [133]:
df_fillna_abd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Age                         300 non-null    int64  
 1   Hospital Number             300 non-null    int64  
 2   pulse                       300 non-null    float64
 3   capillary refill time       300 non-null    float64
 4   temperature of extremities  300 non-null    float64
 5   peristalsis                 300 non-null    float64
 6   abdominal distension        300 non-null    float64
 7   nasogastric reflux          194 non-null    float64
 8   rectal temperature          300 non-null    float64
dtypes: float64(7), int64(2)
memory usage: 21.2 KB


In [142]:
print(df.groupby('Age')['pulse'].median()) # можно предположить, что чем старше лошадь тем пульс выше
age_pulse = df.copy()
age_pulse['pulse'].fillna(df.groupby('Age')['pulse'].transform('median'), inplace=True)
age_pulse['pulse'].describe()

Age
1     60.0
9    124.0
Name: pulse, dtype: float64


count    300.000000
mean      71.386667
std       27.968278
min       30.000000
25%       48.000000
50%       60.000000
75%       88.000000
max      184.000000
Name: pulse, dtype: float64

- объясните, пожалуйста, почему остался столбец nasogastric reflux когда я снова df_fillna_abd.info() (я же вроде его dropнула)
- думаю, что код корявый, слишком много много каких то переназначений df_fillna_te = df_fillna_cap,  df_fillna_cap = df_fillna_t и тд, но по-другому ошибку выдавал " A value is trying to be set on a copy of a slice from a DataFrame"
- как заменить пропущенные значения на mode сразу  в нескольких столбцах, написать функцию?
- можно ли значения одного фрейма сделать заголовками другого? (например, я создам фрейм из названия столбцов dataH.csv и все значения столбца ноль сделать заголовками основного фрейма с симптомами болезни)?
 

In [147]:
horse = pd.read_csv('dataH.csv', header=None)
horse

,0
0,surgery? (операция)
1,Age (возраст)
2,Hospital Number (номер)
3,rectal temperature (ректальная температура)
4,pulse
5,respiratory rate
6,temperature of extremities
7,peripheral pulse
8,mucous membranes
9,capillary refill time
